#### Importing the necessary libraries

In [3]:
from bs4 import BeautifulSoup as bs
import requests as requests

#### Testing out how to get the info box for just one movie

In [ ]:
r = requests.get("https://en.wikipedia.org/wiki/Toy_Story_3")

# Convert to a beautiful soup object
soup = bs(r.content)

In [ ]:
info_box = soup.find(class_ = "infobox vevent")
info_rows = info_box.find_all("tr")

In [ ]:
def replace(string, beforeStr, afterStr) :
    return string.replace(beforeStr, afterStr)

def get_content_value(row_data) :
    if row_data.find("li") :
        result = []
        ## Handle separately
        for li in row_data.find_all("li") :
            clean_content_value = replace(li.get_text(" ", strip=True), "\xa0", " ")
            result.append(clean_content_value)
        return result
    else :
        ## Add td.get_text() directly to movie_info
        return replace(row_data.get_text(" ", strip=True), "\xa0", " ")

    
movie_info = {}
for index, row in enumerate(info_rows) :
    if(index == 0) :
        movie_info['title'] = row.find("th").get_text(" ", strip=True)
    elif index == 1 :
        continue
    else :
        content_key = row.find("th").get_text(" ", strip=True)
        content_value = get_content_value(row.find("td"))
        movie_info[content_key] = content_value

    
movie_info

### Function to get the info box of a movie given its wikipedia url

In [76]:
def replace(string, beforeStr, afterStr) :
        return string.replace(beforeStr, afterStr)

def get_content_value(row_data) :
    if row_data.find("li") :
        result = []
        ## Handle separately
        for li in row_data.find_all("li") :
            clean_content_value = replace(li.get_text(" ", strip=True), "\xa0", " ")
            result.append(clean_content_value)
        return result
    else :
        ## Add td.get_text() directly to movie_info
        return replace(row_data.get_text(" ", strip=True), "\xa0", " ")

def get_movie_info_box(movie_url) :
    r = requests.get(movie_url)

    # Convert to a beautiful soup object
    soup = bs(r.content)
    info_box = soup.find(class_ = "infobox vevent")
    info_rows = info_box.find_all("tr")

    movie_info = {}
    for index, row in enumerate(info_rows) :
        if(index == 0) :
            movie_info['title'] = row.find("th").get_text(" ", strip=True)
        elif index == 1 :
            continue
        else :
            content_key = row.find("th").get_text(" ", strip=True)
            content_value = get_content_value(row.find("td"))
            movie_info[content_key] = content_value
            
    return movie_info

#### Get movie info boxes using the above function

In [ ]:
movie_req = requests.get("https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films")
movie_soup = bs(movie_req.content)

In [101]:
movies = movie_soup.select(".wikitable.sortable i a")
base_path = "https://en.wikipedia.org/"
movie_info_list = []

for index, movie in enumerate(movies):
    # Taking only the first 100 movies
    if(index == 100) :
        break
    try : 
        relative_path = movie['href']
        full_path = base_path + relative_path
        title = movie['title']
        movie_info_list.append(get_movie_info_box(full_path))
        
    except Exception as e :
        print(movie.get_text())
        print(e)

Zorro the Avenger
'NoneType' object has no attribute 'find'
The Sign of Zorro
'NoneType' object has no attribute 'find'


In [103]:
len(movie_info_list)

98

#### Save / Reload movie data

In [104]:
import json

def save_data(title, data) :
    with open(title, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

In [105]:
import json

def load_data(title) :
    with open(title, encoding='utf-8') as f:
        return json.load(f)

In [106]:
save_data("disney_movies.json", movie_info_list)